# HDS5210-2022 Fall Midterm

In the midterm exercise, you're going to use all the programming and data management skills you've developed so far to build a pricing calculator that will calculate how much money should be reimbursed for the visits in a CSV file. To do this, you will need to get allowed amounts (aka rates) from a JSON file, apply some special rules, and then calculate various totals by hospital or by month.

Each step of the midterm will build up to form your final solution. Along the way, I've provided plenty of test cases to make sure you're getting each step correct.

All functions require docstrings with a description and at least one test case.

The midterm is due Monday, October 24th at 11:59 PM CST.

---

## Step 1: Average Rate

With the `/data/negotiated_rates.json` file as input for your first function, read all the `allowed_amount` attributes and calculate an average allowed amount over all rates in the file.

Your function should be named **average_rate()**, take the file's name as it's input parameter, and return a float as the result.

**ROUND YOUR ANSWER TO 2 DECIMAL PLACES**

In [1]:
import json
from statistics import mean 


def average_rate(file):
    """(str) -> float
    The function is designed to calculate the average price of all the listed prices for hospital care and services for those out of network.
    The listing belonds to United HealthCare Services, Inc., a Third-Party Administrator
    
    >>> average_rate('/data/negotiated_rates.json')
    38.67
    """
    price_list = []
    price_rate = open(file)
    price_rate_json = json.load(price_rate)
    out_of_networks = price_rate_json.get("out_of_network")
    
    for out_of_network in out_of_networks:
        allowed_amounts = out_of_network.get("allowed_amounts")
        
        for allowed_amount in allowed_amounts:
            entry = allowed_amount.get("payments").get("allowed_amount")
            price_list.append(entry)
            avg_price = round(mean(price_list), 2)
            
    return avg_price


In [2]:
import doctest
doctest.run_docstring_examples(average_rate, globals(), verbose=True)

Finding tests in NoName
Trying:
    average_rate('/data/negotiated_rates.json')
Expecting:
    38.67
ok


In [3]:
assert(average_rate('/data/negotiated_rates.json') == 38.67)

---

## Step 2: Rate for a Billing Code / Service Code Combination

For the next step, we need to be able to look up the allowed amount for any given billing code / service code combination.

In this data, the billing code represents the service or procedure that was provided. The service code represents the type of site where the service was provided.

Your function should be named **get_rate()** and take three parameters: the JSON file name, the billing code, and the service code. If your code can't find that combination in the file, it should return None.

In [4]:
import json

def get_rate(file, billing_code, service_code):
    """(str, str, str) -> float
    The function is designated to find the price rate of the hospital service using the billing code and the service code as its parameters.
    
    >>> get_rate('/data/negotiated_rates.json','97110','11')
    21.51
    
    >>> get_rate('/data/negotiated_rates.json','99233','21')
    100.35
    
    """
    price_rate = open(file)
    price_rate_json = json.load(price_rate)
    out_of_networks = price_rate_json.get("out_of_network")
    rate_list = []
    
    for out_of_network in out_of_networks:
        allowed_amounts = out_of_network.get("allowed_amounts")
        billing_id = out_of_network.get("billing_code")
        
        for allowed_amount in allowed_amounts:
            service_id = allowed_amount.get("service_code")
            price = allowed_amount.get("payments").get("allowed_amount")
            rate_list.append([billing_id, service_id, price])
    
    for price_list in rate_list:
        if billing_code == price_list[0] and service_code == price_list[1]:
            return price_list[2]
        else:
            result = None
            
    return result
    

In [5]:
import doctest
doctest.run_docstring_examples(get_rate, globals(), verbose=True)

Finding tests in NoName
Trying:
    get_rate('/data/negotiated_rates.json','97110','11')
Expecting:
    21.51
ok
Trying:
    get_rate('/data/negotiated_rates.json','99233','21')
Expecting:
    100.35
ok


In [6]:
assert (get_rate('/data/negotiated_rates.json','G0283','11') == 8.78)
assert (get_rate('/data/negotiated_rates.json','97140','11') == 20.03)
assert (get_rate('/data/negotiated_rates.json','97110','12') == 26.62)

---

## Part 3: Special Rules for Rates

There are some special rules for adjusting rates depending on a patient's age and the day of week.  (These are made up. Not from the real world.) In the next part of the midterm, you'll need to create a function that applies these extra rules to a rate and return the adjusted value.

1. If the day of week is Monday, charge only 75% of the allowed amount.
2. If the patient's age is 65 or higher, charge only 50% of the allowed amount.
3. If's both Monday and the patient's age is 65 or higher, charge only 50% of the allowed amount.
4. If neither of these conditions are true, charge the whole amount.

Your function should be named **get_adjusted_rate()** and take five parameters: file name, billing code, service code, patient age, and visit date.  Your function should return the adjusted rate (based on the rules above) or None if the rate couldn't be found in the file.

Note that your function will take a date in the form `%Y-%m-%d` ([see datetime.strptime()](https://docs.python.org/3/library/datetime.html#datetime.datetime.strptime)) and will need to calculate the [day of week](https://docs.python.org/3/library/datetime.html#datetime.date.weekday).

**ROUND YOUR ANSWER TO 2 DECIMAL PLACES**

In [7]:
import datetime

def get_adjusted_rate(file, billing_code, service_code, pat_age, visit_date):
    """(str, str, str, float, str) -> float
    The function is designated to find the adjusted price rate of the hospital service while adjusting based on the patient's age and the day of the week, using the rate from the function "get_rate".
    
    >>> get_adjusted_rate('/data/negotiated_rates.json','G0283','11',76,'2022-01-03')
    4.39
    
    >>> get_adjusted_rate('/data/negotiated_rates.json','G0283','11',76,'2022-01-08')
    4.39
    
    """
    before_adj_price = get_rate(file, billing_code, service_code)
    
    date_string = datetime.datetime.strptime(visit_date, '%Y-%m-%d')
    date_of_week = datetime.datetime.date(date_string).weekday()

    if date_of_week == 0 and pat_age >= 65:
        adjusted_price = .5 * before_adj_price
    elif date_of_week == 0:
        adjusted_price = .75 * before_adj_price
    elif pat_age >= 65:
        adjusted_price = .5 * before_adj_price
    elif date_of_week != 0 and pat_age < 65:
        adjusted_price = before_adj_price
    else:
        return None
        
    return round(adjusted_price, 2)

In [8]:
import doctest
doctest.run_docstring_examples(get_adjusted_rate, globals(), verbose=True)

Finding tests in NoName
Trying:
    get_adjusted_rate('/data/negotiated_rates.json','G0283','11',76,'2022-01-03')
Expecting:
    4.39
ok
Trying:
    get_adjusted_rate('/data/negotiated_rates.json','G0283','11',76,'2022-01-08')
Expecting:
    4.39
ok


In [9]:
assert (get_adjusted_rate('/data/negotiated_rates.json','G0283','11',22,'2022-01-07') == 8.78)
assert (get_adjusted_rate('/data/negotiated_rates.json','G0283','11',22,'2022-01-03') == 6.58)
assert (get_adjusted_rate('/data/negotiated_rates.json','G0283','11',76,'2022-01-03') == 4.39)
assert (get_adjusted_rate('/data/negotiated_rates.json','G0283','11',76,'2022-01-08') == 4.39)

---

## Step 4: Calculate total payments for a list of visits

Last step, we're going to put your get_adjusted_rate() function to work on a list of visits from a few hospitals in the St. Louis region.

Your last function, **summarize_reimbursement()** needs to read an encounter file and the negotiated rates file, compute an adjusted rate for each encounter (row) in the input file, and return two dictionaries of information:
1. Total expected reimbursement by month
2. Total expected reimbursement by hospital

Your should be able to run your function as `by_month, by_hospital = summarize_reimbursement(visits, negotiated_rates)` and have the two answers below:

**by_month**
```json
{
    '2021-05': 192.38,
    '2021-03': 378.72,
    '2021-07': 277.67,
    '2021-06': 236.53,
    '2021-11': 229.7,
    '2021-10': 234.52,
    '2021-12': 297.87,
    '2021-04': 337.7,
    '2021-09': 160.4,
    '2021-01': 111.91,
    '2021-02': 158.55,
    '2021-08': 152.28
}
```

**by_hospital**
```json
{
    'Missouri Baptist': 514.18,
    'SSM DePaul': 460.02,
    'SLU Hospital': 409.67,
    'Barnes Jewish': 485.7,
    'Mercy Springfield': 518.59,
    'Mercy St. Louis': 380.07
}
```

**Round your totals to 2 decimal places**

**If rates are not found, just ignore them**

In [18]:
import json
import csv
import datetime

def summarize_reimbursement(visits, negotiated_rates):
    """(str, str) -> dict, dict
    The function is designed to calculate the total monetary amount of each visit using the function "get_adjusted_rate",
    then the total is quantitated based on either the month or the hospital.
    
    >>> summarize_reimbursement('/data/visits.csv','/data/negotiated_rates.json')[0]['2021-08']
    152.28
    
    >>> summarize_reimbursement('/data/visits.csv','/data/negotiated_rates.json')[1]['SLU Hospital']
    409.67
    
    >>> summarize_reimbursement('/data/visits.csv','/data/negotiated_rates.json')[1]['Barnes Jewish']
    485.7
    
    """
    
    by_month_total = {}
    by_hospital_total = {}
    price_rate = open(negotiated_rates)
    price_rate_json = json.load(price_rate)
    with open(visits) as visits_file:
        csv_reader = csv.reader(visits_file)
        next (csv_reader, None)
        counter = 0
        max_lines = 200
        for line in csv_reader:
            hospital = line[1]
            pat_age = int(line[3])
            billing_code = line[4]
            service_code = line[5]
            visit_date = line[6]
            file = negotiated_rates
            year_month = line[6][:7]
            
            by_month_total.setdefault(year_month, 0)
            by_hospital_total.setdefault(hospital, 0)
            try:
                price = get_adjusted_rate(file, billing_code, service_code, pat_age, visit_date)
                
                by_month_total[year_month] += price
                by_hospital_total[hospital] += price
            except Exception:
                continue

    for month, total in by_month_total.items():
        by_month_total[month] = round(total, 2)
    for hospital, total in by_hospital_total.items():
        by_hospital_total[hospital] = round(total, 2)
    
    return by_month_total, by_hospital_total
            

In [19]:
summarize_reimbursement('/data/visits.csv','/data/negotiated_rates.json')

({'2021-05': 192.38,
  '2021-03': 378.72,
  '2021-07': 277.67,
  '2021-06': 236.53,
  '2021-11': 229.7,
  '2021-10': 234.52,
  '2021-12': 297.87,
  '2021-04': 337.7,
  '2021-09': 160.4,
  '2021-02': 158.55,
  '2021-01': 111.91,
  '2021-08': 152.28},
 {'Missouri Baptist': 514.18,
  'SSM DePaul': 460.02,
  'SLU Hospital': 409.67,
  'Mercy St. Louis': 380.07,
  'Barnes Jewish': 485.7,
  'Mercy Springfield': 518.59})

In [12]:
import doctest
doctest.run_docstring_examples(summarize_reimbursement, globals(), verbose=True)

Finding tests in NoName
Trying:
    summarize_reimbursement('/data/visits.csv','/data/negotiated_rates.json')[0]['2021-08']
Expecting:
    152.28
ok
Trying:
    summarize_reimbursement('/data/visits.csv','/data/negotiated_rates.json')[1]['SLU Hospital']
Expecting:
    409.67
ok
Trying:
    summarize_reimbursement('/data/visits.csv','/data/negotiated_rates.json')[1]['Barnes Jewish']
Expecting:
    485.7
ok


In [13]:
assert (summarize_reimbursement('/data/visits.csv','/data/negotiated_rates.json')[0]['2021-07'] == 277.67)
assert (summarize_reimbursement('/data/visits.csv','/data/negotiated_rates.json')[0]['2021-03'] == 378.72)
assert (summarize_reimbursement('/data/visits.csv','/data/negotiated_rates.json')[1]['Mercy St. Louis'] == 380.07)
assert (summarize_reimbursement('/data/visits.csv','/data/negotiated_rates.json')[1]['Mercy Springfield'] == 518.59)